In [1]:
# import packages

import pandas as pd
import collections
from matplotlib import pyplot
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
# paramenters

target_scan_species = 9606

In [3]:
# load target tables

target_scan = pd.read_csv('../data/TargetScan.7.2v.miR_Family_info.txt', sep='\t')
encori = pd.read_csv('../data/ENCORI_hg19_all.txt', sep='\t', comment='#')

target_scan = target_scan[target_scan['Species ID'] == target_scan_species]

In [4]:
# create dictionary of key:value as miR_Family:one random representative miR

miRNAid_dict = collections.defaultdict()
family_repr = collections.defaultdict()

for family, group in target_scan.groupby('miR family'):
    for miRNAid in group["MiRBase Accession"].tolist():
        miRNAid_dict[miRNAid] = family
    family_repr[family] = group[["MiRBase Accession", "MiRBase ID"] ].sample(n=1, random_state=1989).values.tolist()[0]

In [5]:
t = group[["MiRBase Accession", "MiRBase ID"] ].sample(n=1, random_state=1989).values.tolist()[0]
t

['MIMAT0000097', 'hsa-miR-99a-5p']

In [6]:
# replace miRNAid in ENCORI df

drop_row = []
for miRNAid, group in encori.groupby('miRNAid'):
    row_indexer = group.index
    try:
        family = miRNAid_dict[miRNAid]
        #miRNAid_family_repr, MiRBase_family_repr = *family_repr[family]
        encori.iloc[row_indexer, [0, 1]] = family_repr[family][0], family_repr[family][1]

    except KeyError:
        drop_row += row_indexer.tolist()


In [7]:
# drop index with no family match
print('original encori shape', encori.shape)
encori.drop(index=drop_row, inplace=True)
print('drop rows', len(drop_row) )
print('after drop encori shape', encori.shape)

original encori shape (2055403, 22)
drop rows 18124
after drop encori shape (2037279, 22)


In [8]:
# save updated encori table

encori.to_csv('encori_unique_mirfamily.tsv', sep='\t', index=False, header=True)

In [15]:
# save train test split
train_df = encori[encori.chromosome != "chr1"]
test_df = encori[encori.chromosome == "chr1"]

In [16]:
train_df.to_csv('encori_train_unique_mirfamily.tsv', sep='\t', index=False, header=True)
test_df.to_csv('encori_test_unique_mirfamily.tsv', sep='\t', index=False, header=True)